In [ ]:
"""Sandbox module."""
from functools import partial

import matplotlib.pyplot as plt
import numpy as np

## Nonparamatric kernel density estimation

In [ ]:
kernels = {
    "Epanechnikov": lambda u: 0.75 * (1 - u**2) * int(abs(u) <= 1),
    "Uniform": lambda u: 0.5 * int(abs(u) <= 1),
    "Triangular": lambda u: (1 - abs(u)) * int(abs(u) <= 1),
}

In [ ]:
def kernel_estimator(x, h, s, kernel_type):
    """Kernel density estimator function."""
    temp_func = np.vectorize(kernels[kernel_type])
    return 1 / (len(s) * h) * sum(temp_func((x - s) / h))

In [ ]:
sample_size = 200
grid_size = 100
grid_ending = 10
bandwidth = 2

sample = np.random.default_rng().normal(size=50)
grid = np.linspace(start=-grid_ending, stop=grid_ending, num=grid_size)

In [ ]:
kernel_estimator_given_sample = partial(kernel_estimator, s=sample)

In [ ]:
values_epa = [
    kernel_estimator_given_sample(x=i, h=bandwidth, kernel_type="Epanechnikov")
    for i in grid
]

In [ ]:
values_uni = [
    kernel_estimator_given_sample(x=i, h=bandwidth, kernel_type="Uniform") for i in grid
]

In [ ]:
plt.plot(grid, values_epa)
plt.plot(grid, values_uni)